![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.png)

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

SDK version: 1.14.0
Turning diagnostics collection on. 


## Initialize workspace

In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: AzureMLCanada
Azure region: canadacentral
Subscription id: 3df1840f-dd4b-4f54-a831-e20536439b3a
Resource group: StatCanDemo


## Create AmlCompute

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=0,
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-10-29T16:17:33.036000+00:00', 'errors': None, 'creationTime': '2020-10-29T16:17:24.001065+00:00', 'modifiedTime': '2020-10-29T16:17:39.729821+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
import os

project_folder = './sklearn-iris'
os.makedirs(project_folder, exist_ok=True)

In [7]:
import shutil

shutil.copy('train_iris.py', project_folder)

'./sklearn-iris/train_iris.py'

### Create an experiment

Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this Scikit-learn tutorial.

In [8]:
from azureml.core import Experiment

experiment_name = 'train_iris'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [2]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn=0.23.2
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


In [10]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

### Configure the training job

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In [11]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train_iris.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment=sklearn_env)

### Submit job

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [12]:
run = experiment.submit(src)

In [13]:
run.wait_for_completion(show_output=True)

RunId: train_iris_1603988266_4e7ccd5c
Web View: https://ml.azure.com/experiments/train_iris/runs/train_iris_1603988266_4e7ccd5c?wsid=/subscriptions/3df1840f-dd4b-4f54-a831-e20536439b3a/resourcegroups/StatCanDemo/workspaces/AzureMLCanada

Streaming azureml-logs/20_image_build_log.txt

2020/10/29 16:17:52 Downloading source code...
2020/10/29 16:17:54 Finished downloading source code
2020/10/29 16:17:54 Creating Docker network: acb_default_network, driver: 'bridge'
2020/10/29 16:17:55 Successfully set up Docker network: acb_default_network
2020/10/29 16:17:55 Setting up Docker configuration...
2020/10/29 16:17:55 Successfully set up Docker configuration
2020/10/29 16:17:55 Logging in to registry: azuremlcanad1510e67a.azurecr.io
2020/10/29 16:17:57 Successfully logged into azuremlcanad1510e67a.azurecr.io
2020/10/29 16:17:57 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/10/29 16:17:57 Scanning for dependencies...
2020/10/29 16

{'runId': 'train_iris_1603988266_4e7ccd5c',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-10-29T16:25:58.943284Z',
 'endTimeUtc': '2020-10-29T16:29:24.288549Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'c1fb2980-9af2-497e-94ad-7258d6c73bfd',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_iris.py',
  'command': [],
  'useAbsolutePath': False,
  'arguments': ['--kernel', 'linear', '--penalty', '1.0'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'environment': {'name': 'sklearn-env',
   'version': 'Autosave_2020-10-29T16:17:48Z_14b6ffe4',
   'python': {'in

### Register the model

In [15]:
model = run.register_model(model_name="iris_model", model_path="outputs")